In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler  
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import traceback

# Configurar o TensorFlow para usar menos memória
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

# --- Função para carregar dados ---
def carregar_dados_ticker(ticker, pasta_dados_tratados):
    """Carrega dados pré-processados APENAS COM FEATURES MICRO."""
    nome_arquivo = f"{ticker}_dados_unificados.csv"
    caminho_arquivo = os.path.join(pasta_dados_tratados, nome_arquivo)

    if not os.path.exists(caminho_arquivo):
        print(f"Erro: Arquivo não encontrado para {ticker} em {caminho_arquivo}")
        return None, None, None

    try:
        print(f"Carregando dados tratados de: {caminho_arquivo}")
        df = pd.read_csv(caminho_arquivo, index_col=0, parse_dates=True)

        print("Colunas originais:", df.columns.tolist())

        # Identificar colunas de preço alvo
        target_col = None
        if 'Preço' in df.columns:
            target_col = 'Preço'
        else:
            print(f"Erro: Nenhuma coluna de preço alvo encontrada para {ticker}")
            return None, None, None

        # Identificar a coluna de feature de preço
        feature_cols = []
        price_feature_col = None
        
        if 'Preço_anterior' in df.columns:
            price_feature_col = 'Preço_anterior'
        else:
            print(f"Aviso: Nenhuma coluna de preço anterior encontrada para {ticker}")
        
        if price_feature_col:
            feature_cols.append(price_feature_col)

        # APENAS features micro
        micro_cols = ['ROA', 'ROE', 'Margem Líquida', 'P/L', 'VP']

        # Adicionar features disponíveis
        for col in micro_cols:
            if col in df.columns:
                feature_cols.append(col)

        if len(feature_cols) < 2:
            print(f"Aviso: Poucas features encontradas para {ticker}: {feature_cols}")

        print(f"Features identificadas para {ticker}: {feature_cols}")
        print(f"Target identificado para {ticker}: {target_col}")

        # Tratar NaNs
        df = df[feature_cols + [target_col]].copy()
        df.dropna(subset=[target_col], inplace=True)
        
        for col in feature_cols:
            df[col] = pd.to_numeric(df[col], errors='coerce')
        
        df[feature_cols] = df[feature_cols].fillna(method='ffill')
        if df[feature_cols].isnull().any().any():
            df[feature_cols] = df[feature_cols].fillna(method='bfill')
        
        if df[feature_cols + [target_col]].isnull().any().any():
            print(f"Aviso: Ainda existem NaNs para {ticker}. Removendo linhas afetadas...")
            df.dropna(inplace=True)

        print(f"Dimensões finais do DataFrame para {ticker}: {df.shape}")
        if df.empty:
            print(f"Erro: DataFrame ficou vazio para {ticker} após limpeza.")
            return None, None, None

        return df, feature_cols, target_col

    except Exception as e:
        print(f"Erro CRÍTICO ao carregar/processar {caminho_arquivo}: {e}")
        traceback.print_exc()
        return None, None, None

# --- Função para criar janelas ---
def criar_janelas_multivariadas_transformer(features_array, target_array, janela):
    """Cria janelas para Transformers SEM diferenciação."""
    X, y = [], []
    
    if len(features_array) <= janela:
        print(f"Aviso: dados insuficientes ({len(features_array)}) para janela ({janela}).")
        return np.array(X), np.array(y)
    
    for i in range(len(features_array) - janela):
        X.append(features_array[i:(i + janela), :])
        y.append(target_array[i + janela])
    
    return np.array(X), np.array(y)

# --- Positional Encoding ---
def positional_encoding(length, depth):
    """Cria positional encoding para capturar ordem temporal."""
    depth = depth / 2
    positions = np.arange(length)[:, np.newaxis]
    depths = np.arange(depth)[np.newaxis, :] / depth
    
    angle_rates = 1 / (10000**depths)
    angle_rads = positions * angle_rates
    
    pos_encoding = np.concatenate([
        np.sin(angle_rads),
        np.cos(angle_rads)
    ], axis=-1)
    
    return tf.cast(pos_encoding, dtype=tf.float32)

# --- Temporal Fusion Transformer Model ---
def build_temporal_fusion_transformer(input_shape, num_heads=4, ff_dim=128, num_transformer_blocks=2, dropout_rate=0.1):
    """Temporal Fusion Transformer."""
    inputs = layers.Input(shape=input_shape)
    
    x = layers.Dense(ff_dim)(inputs)
    positions = positional_encoding(input_shape[0], ff_dim)
    x = x + positions
    
    for _ in range(num_transformer_blocks):
        attention_output = layers.MultiHeadAttention(
            num_heads=num_heads,
            key_dim=ff_dim // num_heads,
            dropout=dropout_rate
        )(x, x)
        attention_output = layers.Dropout(dropout_rate)(attention_output)
        x1 = layers.LayerNormalization(epsilon=1e-6)(x + attention_output)
        
        ffn_output = layers.Dense(ff_dim * 2, activation='relu')(x1)
        ffn_output = layers.Dropout(dropout_rate)(ffn_output)
        ffn_output = layers.Dense(ff_dim)(ffn_output)
        ffn_output = layers.Dropout(dropout_rate)(ffn_output)
        x = layers.LayerNormalization(epsilon=1e-6)(x1 + ffn_output)
    
    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dense(64, activation='relu')(x)
    x = layers.Dropout(dropout_rate)(x)
    x = layers.Dense(32, activation='relu')(x)
    x = layers.Dropout(dropout_rate / 2)(x)
    outputs = layers.Dense(1)(x)
    
    model = keras.Model(inputs=inputs, outputs=outputs, name="TemporalFusionTransformer")
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=0.0005),
        loss='mse',
        metrics=['mae']
    )
    
    return model

# --- Função principal para treinar e avaliar ---
def treinar_avaliar_transformer(ticker, janela, df_full, feature_cols, target_col, 
                                resultados_dir="resultados_transformer_micro"):
    """Treina e avalia o Transformer para um ticker e janela."""
    print(f"\n{'='*60}")
    print(f"TRANSFORMER MICRO - {ticker}, Janela {janela}")
    print(f"{'='*60}")
    
    os.makedirs(resultados_dir, exist_ok=True)
    base_filename = f"{resultados_dir}/{ticker}_Janela_{janela}"
    metrics_path = f"{base_filename}_metrics.csv"
    grafico_path = f"{base_filename}_grafico_teste_final.png"
    previsoes_path = f"{base_filename}_previsoes_teste_final.csv"
    model_path = f"{base_filename}_model.keras"
    
    if os.path.exists(metrics_path):
        print(f"✓ Resultados já existem. Pulando...")
        return
    
    start_date_val = "2020-01-01"
    end_date_val = "2022-12-31"
    start_date_test = "2023-01-01"
    end_date_test = "2023-12-31"
    
    try:
        df_val_train = df_full.loc[start_date_val:end_date_val].copy()
        df_test_final = df_full.loc[start_date_test:end_date_test].copy()
    except KeyError as e:
        print(f"❌ Erro ao dividir dados por data: {e}")
        if not df_full.empty: 
            print(f"Datas disponíveis: {df_full.index.min()} a {df_full.index.max()}")
        return
    
    if df_val_train.empty or df_test_final.empty:
        print(f"❌ Erro: Período treino/val ou teste vazio.")
        return
    
    features_val_train = df_val_train[feature_cols].values
    target_val_train = df_val_train[target_col].values
    
    features_test = df_test_final[feature_cols].values
    target_test = df_test_final[target_col].values
    
    scaler_features = StandardScaler()
    scaler_target = StandardScaler()
    
    scaled_features_val_train = scaler_features.fit_transform(features_val_train)
    target_val_train_reshaped = target_val_train.reshape(-1, 1)
    scaled_target_val_train = scaler_target.fit_transform(target_val_train_reshaped).flatten()
    
    scaled_features_test = scaler_features.transform(features_test)
    target_test_reshaped = target_test.reshape(-1, 1)
    scaled_target_test = scaler_target.transform(target_test_reshaped).flatten()
    
    X_train, y_train = criar_janelas_multivariadas_transformer(
        scaled_features_val_train, 
        scaled_target_val_train, 
        janela
    )
    
    X_test, y_test = criar_janelas_multivariadas_transformer(
        scaled_features_test, 
        scaled_target_test, 
        janela
    )
    
    if len(X_train) == 0 or len(X_test) == 0:
        print(f"❌ Erro: Não foi possível criar janelas.")
        return
    
    print(f"✓ Shape X_train: {X_train.shape}, y_train: {y_train.shape}")
    print(f"✓ Shape X_test: {X_test.shape}, y_test: {y_test.shape}")
    
    val_split_idx = int(0.8 * len(X_train))
    X_train_final, y_train_final = X_train[:val_split_idx], y_train[:val_split_idx]
    X_val, y_val = X_train[val_split_idx:], y_train[val_split_idx:]
    
    input_shape = (janela, X_train.shape[2])
    print(f"✓ Input shape: {input_shape}")
    
    model = build_temporal_fusion_transformer(
        input_shape=input_shape,
        num_heads=4,
        ff_dim=128,
        num_transformer_blocks=2,
        dropout_rate=0.1
    )
    
    print(model.summary())
    
    early_stopping = EarlyStopping(
        monitor='val_loss',
        patience=30,
        restore_best_weights=True,
        verbose=1
    )
    
    reduce_lr = ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=15,
        min_lr=1e-6,
        verbose=1
    )
    
    print(f"\n🚀 Iniciando treinamento...")
    history = model.fit(
        X_train_final, y_train_final,
        validation_data=(X_val, y_val),
        epochs=300,
        batch_size=16,
        callbacks=[early_stopping, reduce_lr],
        verbose=1
    )
    
    model.save(model_path)
    print(f"✓ Modelo salvo em: {model_path}")
    
    predictions_scaled = model.predict(X_test, verbose=0)
    predictions = scaler_target.inverse_transform(predictions_scaled).flatten()
    actual = scaler_target.inverse_transform(y_test.reshape(-1, 1)).flatten()
    
    mae = mean_absolute_error(actual, predictions)
    mse = mean_squared_error(actual, predictions)
    rmse = np.sqrt(mse)
    r2 = r2_score(actual, predictions)
    
    var_real = np.std(actual) / np.mean(actual) if np.mean(actual) != 0 else 0
    var_pred = np.std(predictions) / np.mean(predictions) if np.mean(predictions) != 0 else 0
    prop_var = var_pred / var_real if var_real != 0 else 0
    travado = "Sim" if prop_var < 0.3 else "Não"
    
    print("\n📊 RESULTADOS:")
    print(f"MAE: {mae:.4f} | MSE: {mse:.4f} | RMSE: {rmse:.4f} | R²: {r2:.4f}")
    print(f"Variação Real: {var_real:.4f} | Variação Prevista: {var_pred:.4f}")
    print(f"Previsões travadas? {travado}")
    
    metrics_df = pd.DataFrame([{
        'Ticker': ticker,
        'Janela': janela,
        'Modelo': 'Transformer',
        'MAE': mae,
        'MSE': mse,
        'RMSE': rmse,
        'R2': r2,
        'Var_Real': var_real,
        'Var_Pred': var_pred,
        'Prop_Var': prop_var,
        'Travado': travado,
        'Num_Heads': 4,
        'FF_Dim': 128,
        'Transformer_Blocks': 2
    }])
    metrics_df.to_csv(metrics_path, index=False)
    
    datas_teste = df_test_final.index[janela:janela+len(actual)]
    
    if len(datas_teste) == len(actual):
        previsoes_df = pd.DataFrame({
            'Data': datas_teste,
            'Preço Real': actual,
            'Preço Previsto': predictions
        })
        previsoes_df.to_csv(previsoes_path, index=False)
        
        plt.figure(figsize=(14, 7))
        plt.plot(previsoes_df['Data'], previsoes_df['Preço Real'], 
                label='Real', color='blue', linewidth=2, alpha=0.7)
        plt.plot(previsoes_df['Data'], previsoes_df['Preço Previsto'], 
                label='Previsto (Transformer)', color='red', linestyle='--', linewidth=2, alpha=0.7)
        plt.title(f'Transformer MICRO - {ticker} (Janela {janela})', fontsize=14, fontweight='bold')
        plt.xlabel('Data', fontsize=12)
        plt.ylabel('Preço', fontsize=12)
        plt.legend(fontsize=11)
        plt.grid(alpha=0.3)
        plt.tight_layout()
        plt.savefig(grafico_path, dpi=150)
        plt.close()
    
    print(f"✓ Experimento concluído!\n")
    return metrics_df

# --- Função para rodar experimentos ---
def rodar_experimentos_transformer():
    pasta_dados = r"C:\Users\leona\OneDrive\Área de Trabalho\Machine-Learning---Stock-Prediction\CodigoExperimentos\ExperimentoFeatures\dataframesMicro"
    resultados_dir = r"C:\Users\leona\OneDrive\Área de Trabalho\Machine-Learning---Stock-Prediction\CodigoTransformers\resultados_transformer_micro"
    os.makedirs(resultados_dir, exist_ok=True)
    
    tickers = ["BEEF3.SA", "PETR4.SA", "SOJA3.SA", "GGBR3.SA", "CSNA3.SA", 
               "VALE3.SA", "JBSS3.SA", "BRFS3.SA", "SUZB3.SA"]
    
    janelas = [5, 10, 15, 20]
    
    resultados_consolidados = []
    
    for ticker in tickers:
        df_ticker_full, feature_cols, target_col = carregar_dados_ticker(ticker, pasta_dados)
        
        if df_ticker_full is None:
            print(f"⚠️  Pulando ticker {ticker} devido a erro no carregamento.")
            continue
        
        for janela in janelas:
            try:
                metrics = treinar_avaliar_transformer(
                    ticker, janela, df_ticker_full, 
                    feature_cols, target_col, resultados_dir
                )
                if metrics is not None:
                    resultados_consolidados.append(metrics)
            except Exception as e:
                print(f"❌ Erro ao processar {ticker}, Janela {janela}: {e}")
                traceback.print_exc()
    
    if resultados_consolidados:
        df_resultados = pd.concat(resultados_consolidados, ignore_index=True)
        df_resultados.to_csv(f"{resultados_dir}/resultados_consolidados_transformer.csv", index=False)
        
        print("\n" + "="*60)
        print("📈 RESUMO DOS RESULTADOS")
        print("="*60)
        print(df_resultados.groupby('Ticker')[['MAE', 'RMSE', 'R2']].mean())
        print(f"\n✓ Resultados salvos em: {resultados_dir}/resultados_consolidados_transformer.csv")

if __name__ == "__main__":
    tf.get_logger().setLevel('ERROR')
    
    print("\n" + "="*60)
    print("🚀 TRANSFORMER - EXPERIMENTOS MICRO")
    print("="*60)
    print("\nModelo: Temporal Fusion Transformer")
    print("Features: APENAS MICRO")
    print("Período Treino: 2020-2022")
    print("Período Teste: 2023")
    print("="*60 + "\n")
    
    rodar_experimentos_transformer()
    
    print("\n" + "="*60)
    print("✅ TODOS EXPERIMENTOS CONCLUÍDOS!")
    print("="*60 + "\n")

c:\Users\leona\OneDrive\Área de Trabalho\Machine-Learning---Stock-Prediction\.venv\Lib\site-packages\keras\src\export\tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):
C:\Users\leona\AppData\Local\Temp\ipykernel_35320\3875030829.py:79: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df[feature_cols] = df[feature_cols].fillna(method='ffill')



🚀 TRANSFORMER - EXPERIMENTOS MICRO

Modelo: Temporal Fusion Transformer
Features: APENAS MICRO
Período Treino: 2020-2022
Período Teste: 2023

Carregando dados tratados de: C:\Users\leona\OneDrive\Área de Trabalho\Machine-Learning---Stock-Prediction\CodigoExperimentos\ExperimentoFeatures\dataframesMicro\BEEF3.SA_dados_unificados.csv
Colunas originais: ['Preço', 'ROA', 'ROE', 'Margem Líquida', 'P/L', 'VP', 'Preço_anterior']
Features identificadas para BEEF3.SA: ['Preço_anterior', 'ROA', 'ROE', 'Margem Líquida', 'P/L', 'VP']
Target identificado para BEEF3.SA: Preço
Dimensões finais do DataFrame para BEEF3.SA: (1460, 7)

TRANSFORMER MICRO - BEEF3.SA, Janela 5
✓ Shape X_train: (1090, 5, 6), y_train: (1090,)
✓ Shape X_test: (360, 5, 6), y_test: (360,)
✓ Input shape: (5, 6)


Model: "TemporalFusionTransformer"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 5, 6)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 5, 128)    │        896 │ input_layer[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 5, 128)    │          0 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 5, 128)    │     66,048 │ add[0][0],        │
│ (MultiHeadAttentio… │                   │            │ add[0][0]         │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 5, 128)    │          0 │ multi_head_atten… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_1 (Add)         │ (None, 5, 128)    │          0 │ add[0][0],        │
│                     │                   │            │ dropout_1[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalization │ (None, 5, 128)    │        256 │ add_1[0][0]       │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 5, 256)    │     33,024 │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_2 (Dropout) │ (None, 5, 256)    │          0 │ dense_1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 5, 128)    │     32,896 │ dropout_2[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_3 (Dropout) │ (None, 5, 128)    │          0 │ dense_2[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_2 (Add)         │ (None, 5, 128)    │          0 │ layer_normalizat… │
│                     │                   │            │ dropout_3[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 5, 128)    │        256 │ add_2[0][0]       │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 5, 128)    │     66,048 │ layer_normalizat… │
│ (MultiHeadAttentio… │                   │            │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_5 (Dropout) │ (None, 5, 128)    │          0 │ multi_head_atten… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_3 (Add)         │ (None, 5, 128)    │          0 │ layer_normalizat… │
│                     │                   │            │ dropout_5[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 5, 128)    │        256 │ add_3[0][0]       │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 5, 256)    │     33,024 │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_6 (Dropout) │ (None, 5, 256)    │          0 │ dense_3[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_4 (Dense)     │ (None, 5, 128)    │     32,896 │ dropout_6[0][0]   │
├─────────────────────┼───────────────────┼────────────┼─────────────────

 Total params: 276,225 (1.05 MB)

...

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_31      │ (None, 20, 6)     │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_248 (Dense)   │ (None, 20, 128)   │        896 │ input_layer_31[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_155 (Add)       │ (None, 20, 128)   │          0 │ dense_248[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 20, 128)   │     66,048 │ add_155[0][0],    │
│ (MultiHeadAttentio… │                   │            │ add_155[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_311         │ (None, 20, 128)   │          0 │ multi_head_atten… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_156 (Add)       │ (None, 20, 128)   │          0 │ add_155[0][0],    │
│                     │                   │            │ dropout_311[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 20, 128)   │        256 │ add_156[0][0]     │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_249 (Dense)   │ (None, 20, 256)   │     33,024 │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_312         │ (None, 20, 256)   │          0 │ dense_249[0][0]   │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_250 (Dense)   │ (None, 20, 128)   │     32,896 │ dropout_312[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_313         │ (None, 20, 128)   │          0 │ dense_250[0][0]   │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_157 (Add)       │ (None, 20, 128)   │          0 │ layer_normalizat… │
│                     │                   │            │ dropout_313[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 20, 128)   │        256 │ add_157[0][0]     │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 20, 128)   │     66,048 │ layer_normalizat… │
│ (MultiHeadAttentio… │                   │            │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_315         │ (None, 20, 128)   │          0 │ multi_head_atten… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_158 (Add)       │ (None, 20, 128)   │          0 │ layer_normalizat… │
│                     │                   │            │ dropout_315[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 20, 128)   │        256 │ add_158[0][0]     │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_251 (Dense)   │ (None, 20, 256)   │     33,024 │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼─────────────────

 Total params: 276,225 (1.05 MB)

 Trainable params: 276,225 (1.05 MB)

 Non-trainable params: 0 (0.00 B)

None

🚀 Iniciando treinamento...
Epoch 1/300
54/54 ━━━━━━━━━━━━━━━━━━━━ 5s 20ms/step - loss: 0.2947 - mae: 0.4203 - val_loss: 0.0877 - val_mae: 0.2427 - learning_rate: 5.0000e-04
Epoch 2/300
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - loss: 0.1399 - mae: 0.2949 - val_loss: 0.0509 - val_mae: 0.1789 - learning_rate: 5.0000e-04
Epoch 3/300
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 0.1021 - mae: 0.2487 - val_loss: 0.0821 - val_mae: 0.2435 - learning_rate: 5.0000e-04
Epoch 4/300
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - loss: 0.0891 - mae: 0.2333 - val_loss: 0.2869 - val_mae: 0.4722 - learning_rate: 5.0000e-04
Epoch 5/300
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 0.0874 - mae: 0.2268 - val_loss: 0.0226 - val_mae: 0.1203 - learning_rate: 5.0000e-04
Epoch 6/300
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - loss: 0.0644 - mae: 0.1902 - val_loss: 0.0657 - val_mae: 0.1942 - learning_rate: 5.0000e-04
Epoch 7/300
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: 0.0714 - mae: 0.2022 - val_loss